In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
try:
  # Use the %tensorflow_version magic if in colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

C:\Users\M.Plaban\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\M.Plaban\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\M.Plaban\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\M.Plaban\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWa

In [3]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_hub as hub


from tensorflow.keras import layers

In [4]:
from keras.models import load_model


Using TensorFlow backend.


In [5]:
from keras.preprocessing.image import ImageDataGenerator

In [34]:
IMAGE_RES=224

In [35]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.5,
        horizontal_flip=True)

In [43]:
#mymodel75 batchsize=32
#mymodel70 batchsize=5
#mymodel   batchsize=3
train_generator = train_datagen.flow_from_directory(
    directory=r"C:\Users\M.Plaban\Desktop\mask_detector\data\split_data\train",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=10, 
    class_mode="categorical",
    shuffle=True,
    seed=65
)

Found 826 images belonging to 2 classes.


In [44]:
valid_datagen = ImageDataGenerator(rescale=1./255)

In [45]:
valid_generator = valid_datagen.flow_from_directory(
    directory=r"C:\Users\M.Plaban\Desktop\mask_detector\data\split_data\val",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=10,
    class_mode="categorical",
    shuffle=True,
    seed=65
)

Found 550 images belonging to 2 classes.


In [22]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMAGE_RES, IMAGE_RES, 3))

In [23]:
feature_extractor.trainable = False

In [46]:
model = tf.keras.Sequential([
  feature_extractor,
  layers.Dense(2, activation='softmax')
])

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 1280)              2257984   
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [47]:
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'])

In [48]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

In [49]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5
)

Epoch 1/5
82/82 [==============================] - 20s 245ms/step - loss: 0.1985 - acc: 0.9118 - val_loss: 0.0347 - val_acc: 0.9945
Epoch 2/5
82/82 [==============================] - 18s 216ms/step - loss: 0.0427 - acc: 0.9877 - val_loss: 0.0278 - val_acc: 0.9964
Epoch 3/5
82/82 [==============================] - 17s 213ms/step - loss: 0.0261 - acc: 0.9926 - val_loss: 0.0200 - val_acc: 0.9945
Epoch 4/5
82/82 [==============================] - 17s 210ms/step - loss: 0.0198 - acc: 0.9951 - val_loss: 0.0204 - val_acc: 0.9927
Epoch 5/5
82/82 [==============================] - 17s 212ms/step - loss: 0.0176 - acc: 0.9939 - val_loss: 0.0165 - val_acc: 0.9927


In [50]:
model.save('my_model.h5')